# Fitting to the experimental .xye Time-Of-Flight data

In this notebook we will show how to load a xye file with experimental TOF profile and how to perform a parameter fit.


#### Import Python packages

In [44]:
import numpy as np

import easydiffraction as ed

# Vizualization
import py3Dmol
from bokeh.io import show, output_notebook
from bokeh.plotting import figure

In [ ]:
output_notebook()
FIGURE_WIDTH = 990
FIGURE_HEIGHT = 300

## Sample

#### Show a CIF file content

In [ ]:
cif_fname = 'diamond_phase.cif'

with open(cif_fname, 'r') as f:
    content = f.read()
    
print(content)

#### Create a new TOF job based on the CIF file

In [ ]:
job = ed.Job(type='tof')
job.add_sample_from_file(cif_fname)

Default job type is `pd-cwl-unp-1d-neut` for Powder, Constant Wavelength, Unpolarized, 1D, Neutrons
To switch the job to TOF, we can set it in the constructor, or assing afterwards.
The string could also be `type='pd-tof-unp-1d-neut'` or any susbset of these, like `type='tof-unp-neut'`

In [ ]:
phases = job.phases
phase = phases[0]

print(phases)
print(phase)

#### Visualise the structure

In [ ]:
structure = py3Dmol.view()
structure.addModel(phase.cif, 'cif')
structure.setStyle({'sphere':{'colorscheme':'Jmol','scale':.2},'stick':{'colorscheme':'Jmol','radius': 0.1}})
structure.addUnitCell()
structure.replicateUnitCell(2,2,1)
structure.zoomTo()

## Experiment

#### Read in the data xye file

In [50]:
meas_fname = 'diamond_exp.xye'
job.add_experiment_from_file(meas_fname)

#### Extract the measured data

In [51]:
meas_x = job.experiment.x
meas_y = job.experiment.y

#### Visualize the measured data

In [ ]:
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(meas_x, meas_y, legend_label='Imeas', color='steelblue', line_width=2)
show(fig)

## Analysis

#### The default job is of 1D powder neutron diffraction with time-of-flight type.

In [ ]:
print(f"Job type: {job.type}")

In [ ]:
print(f"Current calculator engine: {job.calculator}")

#### Generate the calculated data

Note that we do not need to pass the `x` values, since they are already known.

In [55]:
calc_y_cryspy = job.calculate_profile()

#### Visualize both the measured and calculated data

In [ ]:
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(meas_x, meas_y, legend_label='Imeas', color='steelblue', line_width=2)
fig.line(meas_x, calc_y_cryspy, legend_label='Icalc (CrysPy)', color='orangered', line_width=2)
show(fig)

#### Set zero shift manually

In [57]:
job.sample.parameters.zero = -18.7

#### Set background points manually

In [58]:
p1 = (meas_x[0], 126)
p2 = (meas_x[-1], 15)
points = [p1, p2]

job.set_background(points)

In [ ]:
calc_y_cryspy = job.calculate_profile(meas_x)

fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(meas_x, meas_y, legend_label='Imeas', color='steelblue', line_width=2)
fig.line(meas_x, calc_y_cryspy, legend_label='Icalc (CrysPy)', color='orangered', line_width=2)
show(fig)

#### Redefine default parameters to get closer to the experimental profile

In [60]:
job.phases[0].scale = 10.0
job.parameters.zero_shift = -18.76
job.parameters.dtt1 = 22602
job.parameters.dtt2 = -8.3
job.parameters.alpha0 = 0.13
job.parameters.beta0 = 0.083
job.parameters.sigma1 = 1.0
job.parameters.ttheta_bank = 90.0

#### Define parameters to optimize

In [61]:
job.phases[0].scale = 10.0
job.phases[0].cell.length_a.fixed = False
job.phases[0].cell.length_b.fixed = False
job.phases[0].cell.length_c.fixed = False
job.pattern.zero_shift.fixed = False
job.parameters.dtt1.fixed = False
job.parameters.dtt2.fixed = False
job.parameters.alpha0.fixed = True
job.parameters.beta0.fixed = True


In [ ]:
calc_y_cryspy = job.calculate_profile(meas_x)

fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(meas_x, meas_y, legend_label='Imeas', color='steelblue', line_width=2)
fig.line(meas_x, calc_y_cryspy, legend_label='Icalc (CrysPy)', color='orangered', line_width=2)
show(fig)

#### Perform the fit with Bumps Simplex method

In [ ]:
print(f"Available minimizers: {job.analysis.available_minimizers}")
print(f"Current minimizer: {job.analysis.current_minimizer}")
job.analysis.current_minimizer = 'Bumps_simplex'
print(f"Current minimizer: {job.analysis.current_minimizer}")

In [ ]:
%%time
job.fit()

Please be aware that this is a long fit, it can take about 15-20 min.

In [ ]:
print("The fit has been successful: {}".format(job.fitting_results.success))
if job.fitting_results.success:    
    print("The gooodness of fit (chi2) is: {}".format(job.fitting_results.reduced_chi))
    print(job.phases[0].scale)
    print(job.pattern.zero_shift)
    print(job.parameters.dtt1)
    print(job.parameters.dtt2)

In [ ]:
calc_y_cryspy = job.calculate_profile(meas_x)

fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(meas_x, meas_y, legend_label='Imeas', color='steelblue', line_width=2)
fig.line(meas_x, calc_y_cryspy, legend_label='Icalc (CrysPy)', color='orangered', line_width=2)
fig.line(meas_x, meas_y-calc_y_cryspy, legend_label='Imeas - Icalc (CrysPy)', color='olivedrab', line_width=2)
show(fig)